In [49]:
# Import libraries
'''
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
'''

# Pretty display for notebooks
'''
%matplotlib inline
'''

# Allows the use of display() for DataFrames
from IPython.display import display 

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
weather = pd.read_csv("../asset/weather.csv")
key = pd.read_csv("../asset/key.csv")
submission_example = pd.read_csv("../asset/sampleSubmission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("weather  data : ", weather.shape)
print("key  data : ", key.shape)
print("submission  data : ", submission_example.shape)

Train data :  (4617600, 4)
Test  data :  (526917, 3)
weather  data :  (20517, 20)
key  data :  (45, 2)
submission  data :  (526917, 2)


In [5]:
train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [6]:
test.tail()

,date,store_nbr,item_nbr
526912,2014-10-26,45,107
526913,2014-10-26,45,108
526914,2014-10-26,45,109
526915,2014-10-26,45,110
526916,2014-10-26,45,111


## Weather data

- Column : 전문용어
    - station_nbr : 기성청 번호
    - date : 날짜
- Temp : 화씨(Fahrenheit)
    - tmax : 최고 온도
    - tmin : 최저 온도
    - tavg : 평균 온도
    - depart : Departure from normal
    - dewpoint : 평균 이슬점
    - wetbulb : 평균 습구?
- Degree Days : Base 65 F
    - Heat : Heating (Season begins with July)
    - Cool : Cooling (Season begins with January)
    - sunrise : Sunrise (Calculated, not observed)
    - sunset : Sunset (Calculated, not observed)
- Significant Weather Types : 날씨 현상
    - codesum : 날씨현상, NO SIGN은 보통인 날을 의미한다.
- Snow/Ice : 인치
    - snowfall : 1/10인치, M = Missing Data, T = Trace
    - preciptotal : 비나, 녹은 눈 인치, M = Missing Data, T = Trace
- Pressure : 인치당 핵토그램(HG)?
    - stnpressure : AVERAGE STATION PRESSURE
    - sealevel : AVERAGE SEA LEVEL PRESSURE
- Wind : SPEED IN MILES PER HOUR, DIRECTION TO TENS OF DEGREES
    - resultspeed : Resultant Wind Speed
    - resultdir : Resultant Wind Direction
    - avgspeed : Average Speed
    
##### _**℃＝(℉－32)/1.8**_

In [21]:
weather.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
20512,16,2014-10-31,53,34,44,M,35,41,21,0,-,-,,M,0.00,29.90,29.99,4.5,03,5.3
20513,17,2014-10-31,59,34,47,M,32,41,18,0,-,-,RA,0.0,0.00,29.72,30.39,9.2,01,9.3
20514,18,2014-10-31,67,49,58,-4,40,50,7,0,0644,1738,,0.0,0.00,29.78,30.28,10.6,36,11.2
20515,19,2014-10-31,45,33,39,-6,24,32,26,0,0624,1646,RA SN,0.1,0.02,29.51,30.24,20.4,34,20.9
20516,20,2014-10-31,68,50,59,M,39,50,6,0,-,-,,0.0,0.00,29.57,30.27,10.5,36,11.2


In [50]:
key.tail()

,store_nbr,station_nbr
40,41,12
41,42,14
42,43,11
43,44,12
44,45,16


In [71]:
key_ = key.sort_values(by = ['station_nbr']).reset_index(drop=True)
key_ = key_.reindex_axis(["station_nbr", "store_nbr"], axis = 1)

In [87]:
key_["store_nbr"] = key_["store_nbr"].astype(str)

In [91]:
key_.groupby("station_nbr").agg(['min', 'max', 'mean'])

ValueError: no results

In [12]:
train.describe().round(2)

,store_nbr,item_nbr,units
count,4617600.00,4617600.00,4617600.00
mean,23.09,56.00,0.99
std,12.95,32.04,9.88
min,1.00,1.00,0.00
25%,12.00,28.00,0.00
50%,23.00,56.00,0.00
75%,34.00,84.00,0.00
max,45.00,111.00,5568.00
